# Control Challenges: 01 Block With Friction

Iacopo Moles  
June 2, 2025

### Parametric Plots

We can convert the set of ODE into a state space representation. The final bode plot of the block position is: @fig-bode

In [1]:
#| label: fig-bode
#| fig-cap: Starting Bode Plot
using DiscretePIDs, ControlSystems, Plots
using Plots
using LinearAlgebra

# System parameters
Ts = 0.02 # sampling time
Tf = 2; #final simulation time
g = 9.81 #gravity
α = 0.0 # slope
μ = 1.0 # friction coefficient
x_0 = -2.0 # starting position
dx_0 = 0.0 # starting velocity
τ = 20.0 # torque constant 

# State Space Matrix
A = [0 1 0
    0 -μ 1
    0 0 -τ
];
B = [0
    0
    τ];
C = [1 0 0
    0 1 0]

sys = ss(A, B, C, 0)      # Continuous

bodeplot(tf(sys))

It has the shape we expect from a motor + friction. Slow pole for the mass + friction and a faster pole for the current & inductance.

Numerically they are:

In [2]:
#| label: fig-pzmap
#| fig-cap: Starting PZ map
display(eigvals(A)) # -20 , -1, 0  
display(pzmap(tf(sys)))

In @fig-pzmap we see that we start with all the pole in the left-half plane, which is good.

We can design a controller with pole placement.

In [3]:
observability(A, C) #OK
controllability(A, B) #OK
ϵ = 0.01;
pp = 15;
p = -2* [pp + ϵ pp - ϵ (pp / 4)];
L = real(place(sys, p, :c));

For some reason pole placement doesn’t work for the observer, I use a Kalman Filter with random fast values.

In [4]:
R1 = diagm([0.001, 0.002, 0.003])
R2 = diagm([0.00005, 0.0002])
K = kalman(sys, R1, R2; direct=false)
cont = observer_controller(sys, L, K; direct=false)

StateSpace{Continuous, Float64}
A = 
    -4.685967915536801       0.714441161776064    0.0
    -1.1422353528957443     -3.272003345550814    1.0
 -6749.998862729541      -1283.5156906468235    -66.5
B = 
 4.685967915536801      0.2855588382239361
 1.1422353528957443     2.272003345550814
 0.0018627295392554767  0.016090646823472696
C = 
 337.49985000000004  64.17498  2.325
D = 
 0.0  0.0

Continuous-time state-space model

We can check the effect of the new controller on the loop

In [5]:
closedLoop = feedback(sys, cont)
clpoles = poles(closedLoop)
setPlotScale("dB")
bodeplot(closedLoop[1, 1], 0.1:40)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path clip-path="url(#clip970)" d="M334.892 843.17 Q331.281 843.17 329.452 846.735 Q327.647 850.277 327.647 857.406 Q327.647 864.513 329.452 868.078 Q331.281 871.619 334.892 871.619 Q338.526 871.619 340.332 868.078 Q342.161 864.513 342.161 857.406 Q342.161 850.277 340.332 846.735 Q338.526 843.17 334.892 843.17 M334.892 839.467 Q340.702 839.467 343.758 844.073 Q346.836 848.656 346.836 857.406 Q346.836 866.133 343.758 870.74 Q340.702 875.323 334.892 875.323 Q329.082 875.323 326.003 870.74 Q322.948 866.133 322.948 857.406 Q322.948 848.656 326.003 844.073 Q329.082 839.467 334.892 839.467 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip970)" d="M1209.45 843.17 Q1205.84 843.17 1204.01 846.735 Q1202.21 850.277 1202.21 857.406 Q1202.21 864.513 1204.01 868.078 Q1205.84 871.619 1209.45 871.619 Q1213.09 871.619 1214.89 868.078 Q1216.72 864.513 1216.72 857.406 Q1216.72 850.277 1214.89 846.735 Q1213.09 843.17 1209.45 843.17 M1209.45 839.467 Q1215.26 839.467 1218.32 844.073 Q1221.4 848.656 1221.4 857.406 Q1221.4 866.133 1218.32 870.74 Q1215.26 875.323 1209.45 875.323 Q1203.64 875.323 1200.56 870.74 Q1197.51 866.133 1197.51 857.406 Q1197.51 848.656 1200.56 844.073 Q1203.64 839.467 1209.45 839.467 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip970)" d="M1231.1 821.663 Q1228.17 821.663 1226.68 824.559 Q1225.22 827.437 1225.22 833.229 Q1225.22 839.003 1226.68 841.9 Q1228.17 844.777 1231.1 844.777 Q1234.05 844.777 1235.52 841.9 Q1237.01 839.003 1237.01 833.229 Q1237.01 827.437 1235.52 824.559 Q1234.05 821.663 1231.1 821.663 M1231.1 818.653 Q1235.82 818.653 1238.31 822.396 Q1240.81 826.12 1240.81 833.229 Q1240.81 840.32 1238.31 844.063 Q1235.82 847.787 1231.1 847.787 Q1226.38 847.787 1223.88 844.063 Q1221.4 840.32 1221.4 833.229 Q1221.4 826.12 1223.88 822.396 Q1226.38 818.653 1231.1 818.653 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip970)" d="M2070.49 843.17 Q2066.88 843.17 2065.05 846.735 Q2063.25 850.277 2063.25 857.406 Q2063.25 864.513 2065.05 868.078 Q2066.88 871.619 2070.49 871.619 Q2074.12 871.619 2075.93 868.078 Q2077.76 864.513 2077.76 857.406 Q2077.76 850.277 2075.93 846.735 Q2074.12 843.17 2070.49 843.17 M2070.49 839.467 Q2076.3 839.467 2079.36 844.073 Q2082.43 848.656 2082.43 857.406 Q2082.43 866.133 2079.36 870.74 Q2076.3 875.323 2070.49 875.323 Q2064.68 875.323 2061.6 870.74 Q2058.55 866.133 2058.55 857.406 Q2058.55 848.656 2061.6 844.073 Q2064.68 839.467 2070.49 839.467 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> 
 
 
 
 
 
 
 <path clip-path="url(#clip970)" d="M226.306 757.656 Q222.695 757.656 220.867 761.221 Q219.061 764.762 219.061 771.892 Q219.061 778.998 220.867 782.563 Q222.695 786.105 226.306 786.105 Q229.941 786.105 231.746 782.563 Q233.575 778.998 233.575 771.892 Q233.575 764.762 231.746 761.221 Q229.941 757.656 226.306 757.656 M226.306 753.952 Q232.117 753.952 235.172 758.559 Q238.251 763.142 238.251 771.892 Q238.251 780.619 235.172 785.225 Q232.117 789.809 226.306 789.809 Q220.496 789.809 217.418 785.225 Q214.362 780.619 214.362 771.892 Q214.362 763.142 217.418 758.559 Q220.496 753.952 226.306 753.952 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip970)" d="M196.723 631.413 Q193.575 631.413 191.723 633.566 Q189.895 635.718 189.895 639.468 Q189.895 643.195 191.723 645.371 Q193.575 647.524 196.723 647.524 Q199.871 647.524 201.7 645.371 Q203.552 643.195 203.552 639.468 Q203.552 635.718 201.7 633.566 Q199.871 631.413 196.723 631.413 M206.006 616.76 L206.006 621.019 Q204.246 620.186 202.441 619.746 Q200.658 619.306 198.899 619.306 Q194.27 619.306 191.816 622.431 Q189.385 625.556 189.038 631.876 Q190.404 629.862 192.464 628.797 Q194.524 627.709 197.001 627.709 Q202.209 627.709 205.219 630.88 Q208.251 634.029 208.251 639.468 Q208.251 644.792 205.103 648.01 Q201.955 651.

From what I understand we are interested in the dotted line in the bottom right. See how flat it is.

We can convert it to the standard PD gain form.

In [6]:
K = L[1];
Ti = 0;
Td = L[2] / L[1];
pid = DiscretePID(; K, Ts, Ti, Td);

We can simulate this with a motor that only outputs the position:

In [7]:
sysreal = ss(A, B, [1 0 0], 0)
ctrl = function (x, t)
    y = (sysreal.C*x)[] # measurement
    d = 0 * [1.0]        # disturbance
    r = 2 * (t >= 0) # reference
    # u = pid(r, y) # control signal
    # u + d # Plant input is control signal + disturbance
    # u =1
    e = x - [r; 0; 0]
    e[3] = 0 # torque not observable, just ignore it in the final feedback
    u = -L * e + d
    u = [maximum([-20 minimum([20 u])])]
end
t = 0:Ts:Tf


res = lsim(sysreal, ctrl, t)

plot(res, plotu=true, plotx=true, ploty=false);ylabel!("u", sp=1);ylabel!("x", sp=2);ylabel!("v", sp=3);ylabel!("T", sp=4)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<path clip-path="url(#clip060)" d="M293.501 327.34 Q289.89 327.34 288.061 330.904 Q286.256 334.446 286.256 341.576 Q286.256 348.682 288.061 352.247 Q289.89 355.789 293.501 355.789 Q297.136 355.789 298.941 352.247 Q300.77 348.682 300.77 341.576 Q300.77 334.446 298.941 330.904 Q297.136 327.34 293.501 327.34 M293.501 323.636 Q299.311 323.636 302.367 328.242 Q305.446 332.826 305.446 341.576 Q305.446 350.303 302.367 354.909 Q299.311 359.492 293.501 359.492 Q287.691 359.492 284.612 354.909 Q281.557 350.303 281.557 341.576 Q281.557 332.826 284.612 328.242 Q287.691 323.636 293.501 323.636 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip060)" d="M338.733 327.34 Q335.121 327.34 333.293 330.904 Q331.487 334.446 331.487 341.576 Q331.487 348.682 333.293 352.247 Q335.121 355.789 338.733 355.789 Q342.367 355.789 344.172 352.247 Q346.001 348.682 346.001 341.576 Q346.001 334.446 344.172 330.904 Q342.367 327.34 338.733 327.34 M338.733 323.636 Q344.543 323.636 347.598 328.242 Q350.677 332.826 350.677 341.576 Q350.677 350.303 347.598 354.909 Q344.543 359.492 338.733 359.492 Q332.922 359.492 329.844 354.909 Q326.788 350.303 326.788 341.576 Q326.788 332.826 329.844 328.242 Q332.922 323.636 338.733 323.636 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip060)" d="M858.232 327.34 Q854.621 327.34 852.792 330.904 Q850.986 334.446 850.986 341.576 Q850.986 348.682 852.792 352.247 Q854.621 355.789 858.232 355.789 Q861.866 355.789 863.672 352.247 Q865.5 348.682 865.5 341.576 Q865.5 334.446 863.672 330.904 Q861.866 327.34 858.232 327.34 M858.232 323.636 Q864.042 323.636 867.097 328.242 Q870.176 332.826 870.176 341.576 Q870.176 350.303 867.097 354.909 Q864.042 359.492 858.232 359.492 Q852.422 359.492 849.343 354.909 Q846.287 350.303 846.287 341.576 Q846.287 332.826 849.343 328.242 Q852.422 323.636 858.232 323.636 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip060)" d="M893.509 324.261 L911.866 324.261 L911.866 328.196 L897.792 328.196 L897.792 336.668 Q898.81 336.321 899.829 336.159 Q900.847 335.974 901.866 335.974 Q907.653 335.974 911.032 339.145 Q914.412 342.316 914.412 347.733 Q914.412 353.312 910.94 356.414 Q907.468 359.492 901.148 359.492 Q898.972 359.492 896.704 359.122 Q894.458 358.752 892.051 358.011 L892.051 353.312 Q894.134 354.446 896.357 355.002 Q898.579 355.557 901.056 355.557 Q905.06 355.557 907.398 353.451 Q909.736 351.344 909.736 347.733 Q909.736 344.122 907.398 342.016 Q905.06 339.909 901.056 339.909 Q899.181 339.909 897.306 340.326 Q895.454 340.742 893.509 341.622 L893.509 324.261 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip060)" d="M1466.16 327.34 Q1462.55 327.34 1460.72 330.904 Q1458.91 334.446 1458.91 341.576 Q1458.91 348.682 1460.72 352.247 Q1462.55 355.789 1466.16 355.789 Q1469.79 355.789 1471.6 352.247 Q1473.42 348.682 1473.42 341.576 Q1473.42 334.446 1471.6 330.904 Q1469.79 327.34 1466.16 327.34 M1466.16 323.636 Q1471.97 323.636 1475.02 328.242 Q1478.1 332.826 1478.1 341.576 Q1478.1 350.303 1475.02 354.909 Q1471.97 359.492 1466.16 359.492 Q1460.35 359.492 1457.27 354.909 Q1454.21 350.303 1454.21 341.576 Q1454.21 332.826 1457.27 328.242 Q1460.35 323.636 1466.16 323.636 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip060)" d="M2020.93 324.261 L2039.29 324.261 L2039.29 328.196 L2025.22 328.196 L2025.22 336.668 Q2026.23 336.321 2027.25 336.159 Q2028.27 335.974 2029.29 335.974 Q2035.08 335.974 2038.46 339.145 Q2041.84 342.316 2041.84 347.733 Q2041.84 353.312 2038.36 356.414 Q2034.89 359.492 2028.57 359.492 Q2026.4 359.492 2024.13 359.122 Q2021.88 358.752 2019.47 358.011 L2019.47 353.312 Q2021.56 354.446 2023.78 355.002 Q2026 355.557 2028.48 355.557 Q2032.48 355.557 2034.82 353.451 Q2037.16 351.344 2037.16 347.733 Q2037.16 344.122 2034.82 342.016 Q2032.48

For more stats:

In [8]:
si = stepinfo(res);
plot(si);title!("Step Response")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<path clip-path="url(#clip150)" d="M206.615 1834 Q203.004 1834 201.175 1837.57 Q199.37 1841.11 199.37 1848.24 Q199.37 1855.34 201.175 1858.91 Q203.004 1862.45 206.615 1862.45 Q210.249 1862.45 212.055 1858.91 Q213.884 1855.34 213.884 1848.24 Q213.884 1841.11 212.055 1837.57 Q210.249 1834 206.615 1834 M206.615 1830.3 Q212.425 1830.3 215.481 1834.9 Q218.559 1839.49 218.559 1848.24 Q218.559 1856.96 215.481 1861.57 Q212.425 1866.15 206.615 1866.15 Q200.805 1866.15 197.726 1861.57 Q194.671 1856.96 194.671 1848.24 Q194.671 1839.49 197.726 1834.9 Q200.805 1830.3 206.615 1830.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip150)" d="M251.846 1834 Q248.235 1834 246.407 1837.57 Q244.601 1841.11 244.601 1848.24 Q244.601 1855.34 246.407 1858.91 Q248.235 1862.45 251.846 1862.45 Q255.481 1862.45 257.286 1858.91 Q259.115 1855.34 259.115 1848.24 Q259.115 1841.11 257.286 1837.57 Q255.481 1834 251.846 1834 M251.846 1830.3 Q257.656 1830.3 260.712 1834.9 Q263.791 1839.49 263.791 1848.24 Q263.791 1856.96 260.712 1861.57 Q257.656 1866.15 251.846 1866.15 Q246.036 1866.15 242.958 1861.57 Q239.902 1856.96 239.902 1848.24 Q239.902 1839.49 242.958 1834.9 Q246.036 1830.3 251.846 1830.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip150)" d="M792.434 1834 Q788.823 1834 786.995 1837.57 Q785.189 1841.11 785.189 1848.24 Q785.189 1855.34 786.995 1858.91 Q788.823 1862.45 792.434 1862.45 Q796.069 1862.45 797.874 1858.91 Q799.703 1855.34 799.703 1848.24 Q799.703 1841.11 797.874 1837.57 Q796.069 1834 792.434 1834 M792.434 1830.3 Q798.245 1830.3 801.3 1834.9 Q804.379 1839.49 804.379 1848.24 Q804.379 1856.96 801.3 1861.57 Q798.245 1866.15 792.434 1866.15 Q786.624 1866.15 783.546 1861.57 Q780.49 1856.96 780.49 1848.24 Q780.49 1839.49 783.546 1834.9 Q786.624 1830.3 792.434 1830.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip150)" d="M827.712 1830.92 L846.068 1830.92 L846.068 1834.86 L831.994 1834.86 L831.994 1843.33 Q833.013 1842.98 834.031 1842.82 Q835.05 1842.64 836.068 1842.64 Q841.855 1842.64 845.235 1845.81 Q848.615 1848.98 848.615 1854.4 Q848.615 1859.97 845.142 1863.08 Q841.67 1866.15 835.351 1866.15 Q833.175 1866.15 830.906 1865.78 Q828.661 1865.41 826.254 1864.67 L826.254 1859.97 Q828.337 1861.11 830.559 1861.66 Q832.781 1862.22 835.258 1862.22 Q839.263 1862.22 841.601 1860.11 Q843.939 1858.01 843.939 1854.4 Q843.939 1850.78 841.601 1848.68 Q839.263 1846.57 835.258 1846.57 Q833.383 1846.57 831.508 1846.99 Q829.656 1847.4 827.712 1848.28 L827.712 1830.92 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip150)" d="M1421.45 1834 Q1417.84 1834 1416.01 1837.57 Q1414.2 1841.11 1414.2 1848.24 Q1414.2 1855.34 1416.01 1858.91 Q1417.84 1862.45 1421.45 1862.45 Q1425.08 1862.45 1426.89 1858.91 Q1428.72 1855.34 1428.72 1848.24 Q1428.72 1841.11 1426.89 1837.57 Q1425.08 1834 1421.45 1834 M1421.45 1830.3 Q1427.26 1830.3 1430.31 1834.9 Q1433.39 1839.49 1433.39 1848.24 Q1433.39 1856.96 1430.31 1861.57 Q1427.26 1866.15 1421.45 1866.15 Q1415.64 1866.15 1412.56 1861.57 Q1409.5 1856.96 1409.5 1848.24 Q1409.5 1839.49 1412.56 1834.9 Q1415.64 1830.3 1421.45 1830.3 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip150)" d="M1997.31 1830.92 L2015.67 1830.92 L2015.67 1834.86 L2001.6 1834.86 L2001.6 1843.33 Q2002.61 1842.98 2003.63 1842.82 Q2004.65 1842.64 2005.67 1842.64 Q2011.46 1842.64 2014.84 1845.81 Q2018.22 1848.98 2018.22 1854.4 Q2018.22 1859.97 2014.74 1863.08 Q2011.27 1866.15 2004.95 1866.15 Q2002.78 1866.15 2000.51 1865.78 Q1998.26 1865.41 1995.86 1864.67 L1995.86 1859.97 Q1997.94 1861.11 2000.16 1861.66 Q2002.38 1862.22 2004.86 1862.22 Q2008.86 1862.22 2011.2 1860.11 Q2013.54 1858.01 2013.54 1854.4 Q2013.54 1850.78 2011.2 1848.68 Q2008.86 1846.57 2004.86 1846.57 Q2002.98 1846.57 2001.11 1846.99 Q

We can also simulate it in a SIMULINK-like environment:

In [9]:
using FMI, DifferentialEquations
fmu = loadFMU(raw"G:\ControlChallengesSolutions\modelica\ControlChallenges\ControlChallenges.BlockOnSlope_Challenges.Examples.WithFriction.fmu") 
simData = simulateME(fmu, (0.0, 5.0); recordValues=["blockOnSlope.x","blockOnSlope.xd","blockOnSlope.usat"]);
unloadFMU(fmu)
plot(simData, states=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<path clip-path="url(#clip240)" d="M251.341 1767.34 Q247.73 1767.34 245.901 1770.9 Q244.096 1774.45 244.096 1781.58 Q244.096 1788.68 245.901 1792.25 Q247.73 1795.79 251.341 1795.79 Q254.975 1795.79 256.781 1792.25 Q258.61 1788.68 258.61 1781.58 Q258.61 1774.45 256.781 1770.9 Q254.975 1767.34 251.341 1767.34 M251.341 1763.64 Q257.151 1763.64 260.207 1768.24 Q263.286 1772.83 263.286 1781.58 Q263.286 1790.3 260.207 1794.91 Q257.151 1799.49 251.341 1799.49 Q245.531 1799.49 242.452 1794.91 Q239.397 1790.3 239.397 1781.58 Q239.397 1772.83 242.452 1768.24 Q245.531 1763.64 251.341 1763.64 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip240)" d="M1173.92 1794.89 L1190.24 1794.89 L1190.24 1798.82 L1168.3 1798.82 L1168.3 1794.89 Q1170.96 1792.13 1175.54 1787.5 Q1180.15 1782.85 1181.33 1781.51 Q1183.57 1778.98 1184.45 1777.25 Q1185.36 1775.49 1185.36 1773.8 Q1185.36 1771.04 1183.41 1769.31 Q1181.49 1767.57 1178.39 1767.57 Q1176.19 1767.57 1173.74 1768.34 Q1171.31 1769.1 1168.53 1770.65 L1168.53 1765.93 Q1171.35 1764.79 1173.81 1764.21 Q1176.26 1763.64 1178.3 1763.64 Q1183.67 1763.64 1186.86 1766.32 Q1190.06 1769.01 1190.06 1773.5 Q1190.06 1775.63 1189.25 1777.55 Q1188.46 1779.45 1186.35 1782.04 Q1185.77 1782.71 1182.67 1785.93 Q1179.57 1789.12 1173.92 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip240)" d="M1647.48 1780.19 Q1650.84 1780.9 1652.71 1783.17 Q1654.61 1785.44 1654.61 1788.77 Q1654.61 1793.89 1651.09 1796.69 Q1647.57 1799.49 1641.09 1799.49 Q1638.92 1799.49 1636.6 1799.05 Q1634.31 1798.64 1631.86 1797.78 L1631.86 1793.27 Q1633.8 1794.4 1636.12 1794.98 Q1638.43 1795.56 1640.95 1795.56 Q1645.35 1795.56 1647.64 1793.82 Q1649.96 1792.08 1649.96 1788.77 Q1649.96 1785.72 1647.81 1784.01 Q1645.68 1782.27 1641.86 1782.27 L1637.83 1782.27 L1637.83 1778.43 L1642.04 1778.43 Q1645.49 1778.43 1647.32 1777.06 Q1649.15 1775.67 1649.15 1773.08 Q1649.15 1770.42 1647.25 1769.01 Q1645.37 1767.57 1641.86 1767.57 Q1639.93 1767.57 1637.74 1767.99 Q1635.54 1768.4 1632.9 1769.28 L1632.9 1765.12 Q1635.56 1764.38 1637.87 1764.01 Q1640.21 1763.64 1642.27 1763.64 Q1647.6 1763.64 1650.7 1766.07 Q1653.8 1768.47 1653.8 1772.59 Q1653.8 1775.46 1652.16 1777.46 Q1650.51 1779.42 1647.48 1780.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip240)" d="M2561.44 1764.26 L2579.8 1764.26 L2579.8 1768.2 L2565.72 1768.2 L2565.72 1776.67 Q2566.74 1776.32 2567.76 1776.16 Q2568.78 1775.97 2569.8 1775.97 Q2575.58 1775.97 2578.96 1779.15 Q2582.34 1782.32 2582.34 1787.73 Q2582.34 1793.31 2578.87 1796.41 Q2575.4 1799.49 2569.08 1799.49 Q2566.9 1799.49 2564.63 1799.12 Q2562.39 1798.75 2559.98 1798.01 L2559.98 1793.31 Q2562.06 1794.45 2564.29 1795 Q2566.51 1795.56 2568.99 1795.56 Q2572.99 1795.56 2575.33 1793.45 Q2577.67 1791.34 2577.67 1787.73 Q2577.67 1784.12 2575.33 1782.02 Q2572.99 1779.91 2568.99 1779.91 Q2567.11 1779.91 2565.24 1780.33 Q2563.38 1780.74 2561.44 1781.62 L2561.44 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip240)" d="M1448.2 1848.7 L1448.2 1854.24 Q1445.72 1852.97 1443.05 1852.33 Q1440.37 1851.7 1437.51 1851.7 Q1433.15 1851.7 1430.95 1853.03 Q1428.79 1854.37 1428.79 1857.04 Q1428.79 1859.08 1430.35 1860.26 Q1431.91 1861.4 1436.62 1862.45 L1438.62 1862.9 Q1444.86 1864.24 1447.47 1866.69 Q1450.11 1869.11 1450.11 1873.47 Q1450.11 1878.43 1446.17 1881.33 Q1442.25 1884.22 1435.38 1884.22 Q1432.51 1884.22 1429.39 1883.65 Q1426.31 1883.11 1422.87 1882 L1422.87 1875.95 Q1426.12 1877.64 1429.27 1878.49 Q1432.42 1879.32 1435.5 1879.32 Q1439.64 1879.32 1441.87 1877.92 Q1444.1 1876.49 1444.1 1873.91 Q1444.1 1871.52 1442.48 1870.25 Q1440.88 1868.98 1435.44 1867.8 L1433.4 1867.32 Q1427.96 1866.18 1425.54 1863.82 Q1423.12 1861.43 1423.12 1857.3 Q1423.12 1852.27 1426.69 1849.53 Q1430.25 1846.79 1436.81 184